# Maum-AI LLama3 8B 모델을 Sagemaker를 통해 g5 인스턴스에 배포하기 

---

## Ref
- [maum-ai/Llama-3-MAAL-8B-Instruct-v0.1](https://huggingface.co/maum-ai/Llama-3-MAAL-8B-Instruct-v0.1)

---

## 실험 환경
- 이 노트북은 SageMaker Studio Code Editor 및 커널 base (Python 3.10.13) 에서 테스트 되었습니다.

---

# 1. 환경 셋업

상위 폴더의 경로를 추가하여 해당 유틸리티, 이미지 폴더를 참조 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)

python path: /home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/40_inference is added
sys.path:  ['/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python310.zip', '/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10', '/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/lib-dynload', '', '/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages', '/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/40_inference']


In [3]:
# install_needed = True
install_needed = False

if install_needed:
    ! pip install sagemaker --upgrade  --quiet
    ! pip list | grep -E "sagemaker"

In [4]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


# 2. HF 파라미터 설정



## 모델 설정

In [5]:
# model id
hf_model_id = 'maum-ai/Llama-3-MAAL-8B-Instruct-v0.1'


## 환경 변수 설정

In [6]:
# instance type
instance_type = "ml.g5.2xlarge"

# Set GPU_NUM
if instance_type == "ml.g5.2xlarge":
    num_gpu = "1"
elif instance_type == "ml.g5.24xlarge":
    num_gpu = "4"
else:
    num_gpu = None

In [7]:
hub = {
	'HF_MODEL_ID': hf_model_id,
	'SM_NUM_GPUS': num_gpu
}

## 추론 도커 이미지 설정

In [8]:
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

image_uri = get_huggingface_llm_image_uri("huggingface",version="2.0.2")
image_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.3.0-tgi2.0.2-gpu-py310-cu121-ubuntu22.04'

## SageMaker Model 의 하위 클래스인 HuggingFaceModel 생성

In [9]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri= image_uri,
	env=hub,
	role=role, 
)

# 3. SageMaker Endpoint 에 배포 

## endpoint_name 생성

In [10]:
from datetime import datetime

def create_ennpoint_name(model_id, instance_type):

    hf_model_id = model_id.split('/')[1]

    instance_type = instance_type.replace('.','-')
    hf_model_id = hf_model_id.replace('.','-')
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d-%H-%M-%S")
    endpoint_name = f"{hf_model_id}-{instance_type}-{formatted_datetime}"

    return endpoint_name

endpoint_name = create_ennpoint_name(hf_model_id, instance_type)
print("endpoint_name: ", endpoint_name)

    

endpoint_name:  Llama-3-MAAL-8B-Instruct-v0-1-ml-g5-2xlarge-2024-06-04-12-17-17


## SageMaker Endpoint 배포

In [11]:
%%time
predictor = huggingface_model.deploy(
	endpoint_name = endpoint_name,
	initial_instance_count=1,
	instance_type= instance_type,
	container_startup_health_check_timeout=300,
  )

-------------!CPU times: user 165 ms, sys: 3.9 ms, total: 169 ms
Wall time: 7min 2s


# 4. 추론

## pay_load 생성

In [12]:
from inference_utils.inference_util import ( print_ww, 
                                            pretty_print_json,
                                            invoke_endpoint_sagemaker
                                       )
                                       
def create_payload_llama_8b(prompt, param):
    # prompt="What is a machine learning?"
    input_data = f"{prompt}"
    pay_load = {"inputs": input_data, "parameters": param}

    # payload_str = json.dumps(pay_load)
    return pay_load
    # return payload_str.encode("utf-8")
    
def llama3_output_parser(response):
    completion = json.loads(response)
    completion = completion[0]
    completion = completion['generated_text']

    return completion


In [13]:
prompt = "Generative AI is"
param = {"do_sample": True, "max_new_tokens": 256}
pay_load = create_payload_llama_8b(prompt, param)


In [14]:
print("## payload: ") 
pretty_print_json(pay_load)


## payload: 
{
    "inputs": "Generative AI is",
    "parameters": {
        "do_sample": true,
        "max_new_tokens": 256
    }
}


## 추론

In [15]:
response = invoke_endpoint_sagemaker(endpoint_name = endpoint_name, 
                         pay_load = pay_load)    




response = llama3_output_parser(response)
print_ww(response)



Generative AI is the process by which computer systems create new content or results autonomously.
It uses deep learning algorithms and neural networks to analyze data and generate new, original
content that can be customized to human preferences based on patterns in the input data.
Generative AI is widely used in many industries, including:
Content creation and publishing
Creating business logos
Developing video game assets
Augmenting product photos
Developing virtual and augmented reality environments
Creating interactive and personalized advertising
Your enterprises can use NetOn's generative AI platform to unlock autonomous content creation and
maximize your business potential. Let's schedule a call to discuss how NetOn can support your
enterprise's growth.


## 한글 추론

In [18]:
prompt = "생성형 AI 는"
param = {"do_sample": True, "max_new_tokens": 256}
pay_load = create_payload_llama_8b(prompt, param)



print("## payload: ") 
# pretty_print_json(pay_load)
pay_load


## payload: 


{'inputs': '생성형 AI 는',
 'parameters': {'do_sample': True, 'max_new_tokens': 256}}

In [19]:
import time
s = time.perf_counter()

response = invoke_endpoint_sagemaker(endpoint_name = endpoint_name, 
                         pay_load = pay_load)    

elapsed_async = time.perf_counter() - s


response = llama3_output_parser(response)




from termcolor import colored

print("## inference esponse: ")                      
print_ww(colored(response, "green"))                         





## inference esponse: 
생성형 AI 는 기계가 스스로 학습하고 인간과 같은 추론 능력을 갖추는 것을 돕는 교육 모델입니다. 생성형 AI 알고리즘은 컴퓨터에게 데이터를 활용하여 환경을 이해하도록
가르치는 것입니다. 개인이 새로운 것을 요구하고, 이 데이터를 기반으로 결과를 생성함으로써 컴퓨터는 경험을 통해 계속 학습하며 복잡한 패턴을 인식하는 방법을 배우게 됩니다. 이를
통해 데이터가 광범위하고 희귀한 경우에도 알고리즘은 새로운 장면, 객체 및 행동을 보는 것이 가능해집니다.
궁금한 점이 있으면 언제든지 문의해 주세요.


## 추론 요약 

In [20]:
prompt = "다음 글을 30자 아내로 요약해줘.\n, 대학교육협의회는 이르면 이번 주 내년도 대학입학 전형시행계획을 심의하고 최종 의대 모집 규모를 승인합니다.\n\
교육부 관계자는 수험생과 학부모를 위해 가능한 빨리 모집 규모를 확정할 필요가 있다며 구체적인 심사 일정을 논의하겠다고 설명했습니다.\
최종 승인 결과에 따라 내년도 의대 정원은 올해보다 천500명 안팎으로 늘어납니다.\
대통령실은 의대 증원이 일단락됐다며 각 대학이 의대 증원을 담은 학칙 개정을 조속히 완료해달라고 당부했습니다.\
동시에 의료계를 향해선 의대 증원 유예와 백지화라는 실현 불가능한 전제조건을 접고 대화하자고 제안했습니다.\
그러나 의료계 반발은 여전합니다.\
당장 의대생 단체는 정부가 귀를 닫고 복귀만을 호소하는 오만한 태도를 보이고 있다고 비판했습니다.\
전공의들 역시 복귀하지 않겠다는 뜻을 분명히 하고 있습니다.\
대한의사협회도 오는 22일 비공개 긴급회의를 열고 의대 교수 등과 구체적인 대응 수단을 논의합니다.\
의료계가 기댔던 의대 증원 집행정지 신청이 기각·각하된 만큼 다음 단계 대응 로드맵을 마련하겠다는 겁니다.\n\n"

param = {"do_sample": True, "max_new_tokens": 512}
pay_load = create_payload_llama_8b(prompt, param)

print("## payload: ") 
# pretty_print_json(pay_load)

pay_load


## payload: 


{'inputs': '다음 글을 30자 아내로 요약해줘.\n, 대학교육협의회는 이르면 이번 주 내년도 대학입학 전형시행계획을 심의하고 최종 의대 모집 규모를 승인합니다.\n교육부 관계자는 수험생과 학부모를 위해 가능한 빨리 모집 규모를 확정할 필요가 있다며 구체적인 심사 일정을 논의하겠다고 설명했습니다.최종 승인 결과에 따라 내년도 의대 정원은 올해보다 천500명 안팎으로 늘어납니다.대통령실은 의대 증원이 일단락됐다며 각 대학이 의대 증원을 담은 학칙 개정을 조속히 완료해달라고 당부했습니다.동시에 의료계를 향해선 의대 증원 유예와 백지화라는 실현 불가능한 전제조건을 접고 대화하자고 제안했습니다.그러나 의료계 반발은 여전합니다.당장 의대생 단체는 정부가 귀를 닫고 복귀만을 호소하는 오만한 태도를 보이고 있다고 비판했습니다.전공의들 역시 복귀하지 않겠다는 뜻을 분명히 하고 있습니다.대한의사협회도 오는 22일 비공개 긴급회의를 열고 의대 교수 등과 구체적인 대응 수단을 논의합니다.의료계가 기댔던 의대 증원 집행정지 신청이 기각·각하된 만큼 다음 단계 대응 로드맵을 마련하겠다는 겁니다.\n\n',
 'parameters': {'do_sample': True, 'max_new_tokens': 512}}

In [21]:
import time
s = time.perf_counter()

response = invoke_endpoint_sagemaker(endpoint_name = endpoint_name, 
                         pay_load = pay_load)    

elapsed_async = time.perf_counter() - s


response = llama3_output_parser(response)




from termcolor import colored

print("## inference esponse: ")                      
print_ww(colored(response, "green"))                         





## inference esponse: 
다음 글을 30자 아내로 요약해줘.
, 대학교육협의회는 이르면 이번 주 내년도 대학입학 전형시행계획을 심의하고 최종 의대 모집 규모를 승인합니다.
교육부 관계자는 수험생과 학부모를 위해 가능한 빨리 모집 규모를 확정할 필요가 있다며 구체적인 심사 일정을 논의하겠다고 설명했습니다.최종 승인 결과에 따라 내년도 의대 정원은
올해보다 천500명 안팎으로 늘어납니다.대통령실은 의대 증원이 일단락됐다며 각 대학이 의대 증원을 담은 학칙 개정을 조속히 완료해달라고 당부했습니다.동시에 의료계를 향해선 의대
증원 유예와 백지화라는 실현 불가능한 전제조건을 접고 대화하자고 제안했습니다.그러나 의료계 반발은 여전합니다.당장 의대생 단체는 정부가 귀를 닫고 복귀만을 호소하는 오만한 태도를
보이고 있다고 비판했습니다.전공의들 역시 복귀하지 않겠다는 뜻을 분명히 하고 있습니다.대한의사협회도 오는 22일 비공개 긴급회의를 열고 의대 교수 등과 구체적인 대응 수단을
논의합니다.의료계가 기댔던 의대 증원 집행정지 신청이 기각·각하된 만큼 다음 단계 대응 로드맵을 마련하겠다는 겁니다.

요약 하나: 대학교육협의회는 내년도 의대 입학 전형 시행 계획과 의대 모집 규모 확정을 앞두고 있다. 이번 확대는 기존의 오류와 지연을 설명하기 하였으며, 의사협회와의 대화 제안에
대해 의료계는 아직 반발하고 있다.


# 5. 엔드포인트 삭제

In [22]:
def delete_endpoint_model(endpoint_name,llm_model ):
    sess.delete_endpoint(endpoint_name)
    sess.delete_endpoint_config(endpoint_name)
    llm_model.delete_model()

delete_endpoint_model(endpoint_name,huggingface_model)

